In [1]:
import glob
import os
import logging
import sys

import numpy as np
import tifffile
from skimage import io
import skimage as ski
from cellpose import models
import napari
import torch

Set up logging so the cellpose will print information during training

In [2]:
viewer = napari.Viewer()

<div style="width:80ch; margin:10;">
Use glob to find tiff images in a folders. These are the images we need to create manual labels of so a custom training model can be built.
</div>

In [10]:
images = sorted(glob.glob("files/Training/sample_images/*.tif"))
len(images)

23

In [11]:
fname = images[6]
x = tifffile.imread(fname)
x.shape

(2048, 2048)

In [12]:
viewer.layers.clear()
viewer.add_image(x)

<Image layer 'x' at 0x2695e3a9cf0>

Let see how well cellpose does on this image.
- get a model just like before
- run eval

In [13]:
if sys.platform == 'darwin':
    d = torch.device('mps')
    model = models.Cellpose(gpu=False, device=d, model_type='cyto2')
else:
    # change gpu=True if on windows, and get rid of device
    model = models.Cellpose(gpu=True, model_type='cyto2')

In [14]:

masks, flows, _, _ = model.eval(x, channels=[0, 0], diameter=180)

In [15]:
viewer.add_labels(masks)

<Labels layer 'masks' at 0x26972109180>

There are mistakes in the mask, so labels will need to be created manually.
Let's look at the annotation tools in the label layer.


....


# Manual annotation

The napari viewer has a lot of useful tools for annotating images.  We will always be using a labels layer to do so.  You can create one next to the trash can icon (looks like a luggage tag).  

Once a labels layer is selected, you paint individual objects with the paint brush/fill tools.  The way we denote one object as being distinct from another is by making sure we paint with a different color on each object.  The "label" slider at the top of the bar tells us which object we are currently painting.  By pressing the "m" key on our keyboard we can automatically select the next unused index:  very handy for moving on to a next object.  

The other drawing tools are above the "label" slider:  erasure, paintbrush, fill, eye dropper, navigate.  By mousing over them you can see that each has a keyboard shortcut key (the numeric keys 1, 2, 3, 4 etc.)

Sometimes we have to start from scratch:  a completely blank labels layer where we paint each and every object in the image, but frequently we can piggyback on something that is already halfway there like thresholding or cellpose itself.

### Label images for training
- View image
- model.eval the image
- add labels
- edit the labels
- get labels out of the viewer
- create a stack with the image and the mask
- save to file

# Manual annotation (assisted)

# Exporting our label image for training

Regardless whether you used purely manual or assisted manual, you will need to save your labels layer to be used later with cellpose for training.  Assuming the most recent (top) layer of napari is our labels layer that we want to save, we can access its data with viewer.layers[-1].data

In [16]:
masks = viewer.layers[-1].data


((2, 2048, 2048), dtype('uint16'))

Next, as in last class, we will combine the two single channel images into one double channel image.

In [ ]:
tx = np.stack([x, masks])
tx.shape, tx.dtype

In [ ]:
bn = os.path.basename(fname)
tifffile.imwrite(f"Data/Training/for_training/{bn}", tx.astype(np.uint16))